In [207]:
import pandas as pd
import numpy as np
df = pd.read_csv('womenshoes.zip')

In [160]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateAdded',
       'dateUpdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageURLs', 'isbn', 'keys', 'manufacturer',
       'manufacturerNumber', 'merchants', 'name', 'prices.amountMin',
       'prices.amountMax', 'prices.availability', 'prices.color',
       'prices.condition', 'prices.count', 'prices.currency',
       'prices.dateAdded', 'prices.dateSeen', 'prices.flavor', 'prices.isSale',
       'prices.merchant', 'prices.offer', 'prices.returnPolicy',
       'prices.shipping', 'prices.size', 'prices.source', 'prices.sourceURLs',
       'prices.warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceURLs', 'upc', 'vin', 'websiteIDs', 'weight', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51'],
      dtype='object')

In [161]:
df.shape

(33801, 52)

### Missing values

In [208]:
def nas_sorted(df):
    return df.isnull().sum().sort_values(ascending = False)
sorted_nas = nas_sorted(df)
sorted_nas

prices.count           33801
prices.warranty        33801
Unnamed: 49            33801
Unnamed: 48            33801
count                  33801
websiteIDs             33801
vin                    33801
flavors                33801
isbn                   33801
Unnamed: 50            33801
prices.flavor          33801
prices.source          33801
Unnamed: 51            33800
prices.availability    33494
dimension              32624
weight                 32538
prices.returnPolicy    32427
asins                  31379
prices.shipping        29853
reviews                29613
prices.offer           27003
sizes                  25199
manufacturer           24522
ean                    24023
upc                    23077
prices.condition       21786
merchants              18419
prices.size            18410
prices.color           16380
quantities             15311
colors                 13957
skus                   11666
imageURLs              11492
features                6620
descriptions  

In [209]:
pd.set_option('display.max_rows',100)

### Drop columns with all missing values

In [210]:
df.dropna(axis='columns', how='all', inplace=True)

In [211]:
df.dropna(thresh=27, inplace = True)

In [212]:
df.dropna(axis='columns', how='all', inplace=True)

In [213]:
nas_sorted(df)

weight                 6484
prices.availability    6426
asins                  6245
dimension              6127
prices.returnPolicy    5532
reviews                5244
prices.shipping        5170
quantities             4511
prices.offer           4065
prices.size            3724
prices.color           3639
manufacturer           3081
skus                   2678
ean                    2040
prices.condition       1864
upc                    1756
sizes                  1597
descriptions            965
features                844
prices.merchant         495
colors                  462
manufacturerNumber      193
merchants               149
imageURLs                 8
brand                     5
dateUpdated               0
dateAdded                 0
categories                0
sourceURLs                0
prices.amountMax          0
keys                      0
name                      0
prices.amountMin          0
prices.currency           0
prices.dateAdded          0
prices.dateSeen     

### Find outliers 

In [499]:
np.arange(-3, 4, 1)

array([-3, -2, -1,  0,  1,  2,  3])

In [498]:
from scipy import stats

def percentiles(column):
    z_scores = stats.zscore(column)
    # segment based on number of standard deviations away from the mean     
    hist, bin_edges = np.histogram(z_scores, bins=np.arange(-3, 4, 1), density=True)
    return np.stack((hist, bin_edges[1:]))

In [15]:
percentiles(df['prices.amountMax'])

array([[ 0.        ,  0.        ,  0.85676802,  0.07007254,  0.02484951,
         0.04830992],
       [-2.        , -1.        ,  0.        ,  1.        ,  2.        ,
         3.        ]])

#### for the prices.amountMax column, we see there are some outliers between  z-score 2 and 3.

In [500]:
def too_many_outliers(column, threshold = .05):
    #  expected .021 if normal distribution
    z_less_neg_two = percentiles(column)[0, 0]
    z_gt_two = percentiles(column)[0, -1]
    if z_less_neg_two > threshold or z_gt_two > threshold:
        return np.hstack((column.name, z_less_neg_two, z_gt_two))
    else:
        False

In [501]:
def outlier_columns(df, threshold = .05):
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    outlier_columns = np.array([too_many_outliers(df[column]) for column in numeric_columns])
    return np.array([column for column in outlier_columns if column is not None])

In [502]:
def select_outliers(column, upper_tail = True):
    if upper_tail:
        return column[stats.zscore(column) > 2]
    else:
        return column[stats.zscore(column) < -2]

In [19]:
outlier_columns(df)

C:\Users\51759\Anaconda3\lib\site-packages\numpy\lib\histograms.py:893: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


array([], dtype=float64)

### No outliers? Really? Let's check out.

In [20]:
# numerical columns we have
df.select_dtypes(include=['float64', 'int64']).columns

Index(['ean', 'prices.amountMin', 'prices.amountMax', 'upc'], dtype='object')

In [16]:
# from the nan amount of columns we checked before, we can figure out that only 'prices.amountMin' and 'prices.amountMax' are
# valuable data with no missing value. And the rest of them have over  two third of missing values, or even total missing ones.
percentiles(df['ean'])

array([[nan, nan, nan, nan, nan, nan],
       [-2., -1.,  0.,  1.,  2.,  3.]])

In [21]:
percentiles(df['prices.amountMin'])

array([[ 0.        ,  0.        ,  0.85892885,  0.06791171,  0.02484951,
         0.04830992],
       [-2.        , -1.        ,  0.        ,  1.        ,  2.        ,
         3.        ]])

In [22]:
percentiles(df['upc'])

array([[nan, nan, nan, nan, nan, nan],
       [-2., -1.,  0.,  1.,  2.,  3.]])

In [17]:
#select_outliers(df['prices.amountMax']).value_counts()

### Currency convert

In [214]:
# there are many non-US dollars values in currency column, we need to convert them into US dollars.
df['prices.currency'].value_counts()

USD    6663
CAD      37
AUD       1
Name: prices.currency, dtype: int64

In [215]:
df_currency_nonUSD = list(df[df['prices.currency'] != 'USD'].index)

In [216]:
df_nonUSD = df[df['prices.currency'] != 'USD']

In [217]:
df_nonUSD[:3]

,id,asins,brand,categories,colors,dateAdded,dateUpdated,descriptions,dimension,ean,...,prices.shipping,prices.size,prices.sourceURLs,quantities,reviews,sizes,skus,sourceURLs,upc,weight
1655,AVpe7vNAilAPnD_xRMsN,"B011SAUSXO,B011GCU5QY",Alfani,"Shoes,Women's Shoes,Clothing,All Women's Shoes...",Black,2015-08-14T22:32:14Z,2016-11-04T03:10:32Z,"[{""dateSeen"":[""2016-11-04T03:10:32Z""],""sourceU...",NaN,6.083560e+11,...,CAD 19.99,NaN,http://www.amazon.ca/Alfani-Lyrra-Womens-Black...,NaN,NaN,"8.5,7,8",NaN,https://www.walmart.com/ip/Alfani-Lyrra-Peep-T...,6.083560e+11,905 grams
1657,AVpe7vNAilAPnD_xRMsN,"B011SAUSXO,B011GCU5QY",Alfani,"Shoes,Women's Shoes,Clothing,All Women's Shoes...",Black,2015-08-14T22:32:14Z,2016-11-04T03:10:32Z,"[{""dateSeen"":[""2016-11-04T03:10:32Z""],""sourceU...",NaN,6.083560e+11,...,CAD 19.99,NaN,http://www.amazon.ca/Alfani-Lyrra-Womens-Black...,NaN,NaN,"8.5,7,8",NaN,https://www.walmart.com/ip/Alfani-Lyrra-Peep-T...,6.083560e+11,905 grams
1659,AVpe7vNAilAPnD_xRMsN,"B011SAUSXO,B011GCU5QY",Alfani,"Shoes,Women's Shoes,Clothing,All Women's Shoes...",Black,2015-08-14T22:32:14Z,2016-11-04T03:10:32Z,"[{""dateSeen"":[""2016-11-04T03:10:32Z""],""sourceU...",NaN,6.083560e+11,...,CAD 19.99,NaN,http://www.amazon.ca/Alfani-Lyrra-Womens-Black...,NaN,NaN,"8.5,7,8",NaN,https://www.walmart.com/ip/Alfani-Lyrra-Peep-T...,6.083560e+11,905 grams


In [218]:
len(df_nonUSD['prices.currency'])

38

In [219]:
import requests
# Where USD is the base currency you want to use
url = 'https://api.exchangerate-api.com/v4/latest/USD'

# Making our request
response = requests.get(url)
currency_data = response.json()

# Your JSON object
#currency_data

In [220]:
def convert_nonUSD_to_USD(price_col, curr_col, ratio_col):
    return price_col / [ratio_col.get(curr_col.iloc[i]) for i in range(len(curr_col))]

In [221]:
convert_nonUSD_to_USD(df_nonUSD['prices.amountMax'], df_nonUSD['prices.currency'], currency_data['rates'])[:2]

1655    61.263804
1657    31.002510
Name: prices.amountMax, dtype: float64

In [222]:
def convert_nonUSD_to_USD_1(df, price_col, curr_col, ratio_col):
    return df[price_col] / [ratio_col.get(df[curr_col].iloc[i]) for i in range(len(df[curr_col]))]

In [223]:
new_price_max = convert_nonUSD_to_USD_1(df_nonUSD, 'prices.amountMax', 'prices.currency', currency_data['rates'])

In [224]:
def convert_currency_df(df, price_set, curr_col, ratio_col):
    df_all_USD = pd.DataFrame()
    for i in range(len(price_set)):
        df_all_USD[price_set[i]] = convert_nonUSD_to_USD_1(df, price_set[i], curr_col, ratio_col)
    return df_all_USD

In [225]:
price_set = ['prices.amountMin', 'prices.amountMax']

In [226]:
price_cols_all_USD = convert_currency_df(df, price_set, 'prices.currency', currency_data['rates'])

In [227]:
price_cols_all_USD.loc[1655,:]

prices.amountMin    61.263804
prices.amountMax    61.263804
Name: 1655, dtype: float64

In [228]:
df[price_set].loc[1655,:]

prices.amountMin    81.0
prices.amountMax    81.0
Name: 1655, dtype: float64

In [229]:
# we can see index = 1655 is a CAD currency, and the value in price_cols_all_USD is after converting.
df[price_set] = price_cols_all_USD

In [230]:
df[price_set].loc[1655,:]
# we can see it's changed after assigning new values

prices.amountMin    61.263804
prices.amountMax    61.263804
Name: 1655, dtype: float64

In [231]:
nas_sorted(df)

weight                 6484
prices.availability    6426
asins                  6245
dimension              6127
prices.returnPolicy    5532
reviews                5244
prices.shipping        5170
quantities             4511
prices.offer           4065
prices.size            3724
prices.color           3639
manufacturer           3081
skus                   2678
ean                    2040
prices.condition       1864
upc                    1756
sizes                  1597
descriptions            965
features                844
prices.merchant         495
colors                  462
manufacturerNumber      193
merchants               149
imageURLs                 8
brand                     5
dateUpdated               0
dateAdded                 0
categories                0
sourceURLs                0
prices.amountMax          0
keys                      0
name                      0
prices.amountMin          0
prices.currency           0
prices.dateAdded          0
prices.dateSeen     

In [232]:
df[:2].T

,10,11
id,AVpe_0ob1cnluZ0-bnWr,AVpe_0ob1cnluZ0-bnWr
asins,NaN,NaN
brand,BEAUTIFEET,BEAUTIFEET
categories,"Shoes,Women's Shoes,Clothing,All Women's Shoes","Shoes,Women's Shoes,Clothing,All Women's Shoes"
colors,"Multicolor,MULTI","Multicolor,MULTI"
dateAdded,2016-11-04T03:11:11Z,2016-11-04T03:11:11Z
dateUpdated,2016-11-04T03:11:11Z,2016-11-04T03:11:11Z
descriptions,"[{""dateSeen"":[""2016-11-04T03:11:11Z""],""sourceU...","[{""dateSeen"":[""2016-11-04T03:11:11Z""],""sourceU..."
dimension,6.0 in x 6.0 in x 1.0 in,6.0 in x 6.0 in x 1.0 in
ean,9.65065e+10,9.65065e+10


In [233]:
df['quantities'].value_counts()

[{"dateSeen":["2017-03-25T08:57:49.471Z","2017-02-06T20:05:11.986Z"],"sourceURLs":["https://www.overstock.com/Clothing-Shoes/Journee-Collection-Womens-Shelley-3-Slouch-Mid-Calf-Microsuede-Boot/4233538/product.html"],"value":20}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                 57
[{"dateSeen":["2017-03-25T12:03:15.129Z","2017-02-08T15:18:51.258Z"],"sourceURLs":["https://www.overstock.com/Clothing-Shoes/Womens-Ros-Hommerson-Delta-Penny-Loafer-Coffee-Leather/12336563/product.html"],"value":12}]                                                                                                        

we can see from the above cell, the information includes many other infor except quantities, e.g. dateseen, sourceURLs,etc, which can also seen in others columns.
And for the quantities value, we can extract later.

In [234]:
df.dtypes

id                      object
asins                   object
brand                   object
categories              object
colors                  object
dateAdded               object
dateUpdated             object
descriptions            object
dimension               object
ean                    float64
features                object
imageURLs               object
keys                    object
manufacturer            object
manufacturerNumber      object
merchants               object
name                    object
prices.amountMin       float64
prices.amountMax       float64
prices.availability     object
prices.color            object
prices.condition        object
prices.currency         object
prices.dateAdded        object
prices.dateSeen         object
prices.isSale             bool
prices.merchant         object
prices.offer            object
prices.returnPolicy     object
prices.shipping         object
prices.size             object
prices.sourceURLs       object
quantiti

In [235]:
df['prices.offer'].value_counts()
# in this column, it mainly includes reduced, clearance, rollback, winningbid

winning bid             180
REDUCED                  45
ROLLBACK                 33
REDUCED USD 40.01        24
REDUCED USD 28.01        24
CLEARANCE USD 19.01      24
CLEARANCE USD 55.01      22
CLEARANCE USD 42.01      21
REDUCED USD 36.01        20
REDUCED USD 32.01        19
CLEARANCE USD 45.01      19
REDUCED USD 25.01        19
REDUCED USD 52.01        19
REDUCED USD 30.01        19
CLEARANCE USD 62.01      18
REDUCED USD 20.01        18
REDUCED USD 12.01        17
REDUCED USD 27.01        17
CLEARANCE USD 53.01      17
REDUCED USD 19.01        17
CLEARANCE USD 50.01      17
CLEARANCE USD 46.01      16
CLEARANCE                16
CLEARANCE USD 75.01      16
CLEARANCE USD 54.01      16
CLEARANCE USD 36.01      16
REDUCED USD 23.01        16
CLEARANCE USD 57.01      15
CLEARANCE USD 63.01      15
REDUCED USD 18.01        15
REDUCED USD 47.01        15
REDUCED USD 35.01        15
CLEARANCE USD 61.01      14
REDUCED USD 54.01        14
CLEARANCE USD 65.01      14
REDUCED USD 29.01   

### Brand column fixing

In [236]:
nas_sorted(df)

weight                 6484
prices.availability    6426
asins                  6245
dimension              6127
prices.returnPolicy    5532
reviews                5244
prices.shipping        5170
quantities             4511
prices.offer           4065
prices.size            3724
prices.color           3639
manufacturer           3081
skus                   2678
ean                    2040
prices.condition       1864
upc                    1756
sizes                  1597
descriptions            965
features                844
prices.merchant         495
colors                  462
manufacturerNumber      193
merchants               149
imageURLs                 8
brand                     5
dateUpdated               0
dateAdded                 0
categories                0
sourceURLs                0
prices.amountMax          0
keys                      0
name                      0
prices.amountMin          0
prices.currency           0
prices.dateAdded          0
prices.dateSeen     

In [237]:
df.brand.value_counts(normalize = True)

Ralph Lauren            0.124403
Nike                    0.033751
Nine West               0.025388
Trotters                0.022252
SoftWalk                0.020012
Easy Spirit             0.017622
New Balance             0.016428
Journee Collection      0.015084
Nature Breeze           0.014486
MUK LUKS                0.014038
Brinley Co.             0.013889
CAPE ROBBIN             0.012843
Propet                  0.012843
Dyeables                0.012694
TOMS                    0.012694
Faded Glory             0.012097
Nina                    0.011947
Dr. Scholl's            0.011499
Ros Hommerson           0.010753
Naturalizer             0.010753
Skechers                0.010454
VANS                    0.009857
Daniel Green            0.009857
Alfani                  0.008363
MICHAEL Michael Kors    0.008214
Dansko                  0.008214
Bearpaw                 0.008065
Nomad                   0.007467
Touch Ups               0.007467
Qupid                   0.007019
Unbranded 

#### we'll set a bar to keep only some brands and set others to a group 'other' later

### Category column fixing

In [238]:
import re
df['categories'].value_counts()

Shoes,Women's Shoes,Clothing,All Women's Shoes                                                                  996
Clothing,Shoes,Women's Shoes,All Women's Shoes                                                                  951
Shoes,Clothing,Women's Shoes,All Women's Shoes                                                                  768
Women,Shoes,Boots                                                                                               391
Booties,Shoes,Women's Shoes,Clothing,All Women's Shoes                                                          197
Women,Shoes                                                                                                     182
Boots On Sale,Shoes,Women's Shoes,Boots,Clothing,All Women's Shoes                                              132
Sandals,Women,Shoes                                                                                             117
Loafers,Shoes,Women's Shoes,Clothing,All Women's Shoes                  

In [239]:
category_set = ['Sandals', 'Heels', 'Slip-ons', 'Boots', 'Booties', 
                'Wedges', 'Flats', 'Athletic', 'Sneakers', 'Loafers', 'Slippers']

In [240]:
def return_keyword(keys, text):
    for i in range(len(keys)):
        if text.find(keys[i]) != -1:
            return keys[i]
    return 'other'

In [241]:
def match_category(column, category_set):
    new_category = []
    for i in range(len(column)):
        new_category.append(return_keyword(category_set, column.iloc[i]))
    return new_category

In [242]:
range(len(df['categories']))

range(0, 6701)

In [243]:
df['categories'] = match_category(df['categories'], category_set)

In [244]:
df['categories'].value_counts(normalize = True)

other       0.501865
Boots       0.150575
Sandals     0.085360
Athletic    0.063125
Slip-ons    0.050589
Slippers    0.034174
Heels       0.033428
Booties     0.029399
Loafers     0.022982
Wedges      0.012088
Flats       0.008805
Sneakers    0.007611
Name: categories, dtype: float64

### Merchant column fixing

In [199]:
df['prices.merchant'].value_counts()
# A lot of Walmart.com are distributed under many different names, then we should rename them, at least for Walmart.

ApparelSave - Walmart.com                           2141
Overstock.com                                       1823
www.ralphlauren.com                                  831
Shoes.com                                            259
Walmart.com                                          181
ShoeFabs - Walmart.com                               133
shoezoo - Walmart.com                                 93
Virtual Exchanges Inc - Walmart.com                   91
Shoebuy.com - Walmart.com                             59
AreaTrend - Walmart.com                               49
BESTON SHOES - Walmart.com                            40
ShoeMetro CA                                          35
1 ShoeSmart - Walmart.com                             32
Costume Kingdom - Walmart.com                         28
Walmart store - Walmart.com                           26
Purchase Pals LLC - Walmart.com                       21
Shoenerds - Walmart.com                               21
UnbeatableSale - Walmart.com   

In [113]:
# a lot of Walmart in different names, we need to coerce names to walmart.
merchant_set = ['Walmart', 'Overstock', 'ralphlauren', 'Shoes']
# For these high-frequency merchant, I would like to keep their names, and for other brands I set them to 'Others'.

In [351]:
have_Walmart = df['prices.merchant'].str.contains('Walmart')
for i in range(len(df['prices.merchant'])):
    if(have_Walmart[i] == True):
        df['prices.merchant'][i] = 'Walmart'

C:\Users\51759\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [347]:
# this method is NOT NOT NOT efficient,and time complexity is high, needed to improve!!!
for i in range(len(df['prices.merchant'])):
    if(df['prices.merchant'].str.contains('Walmart')[i] == True):
        df['prices.merchant'][i] = 'Walmart'
    

C:\Users\51759\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [352]:
df['prices.merchant'].value_counts()

Overstock.com                                    5776
Walmart                                          4974
www.ralphlauren.com                              1417
Shoes.com                                         392
ShoeMetro CA                                       58
Amazon.com                                         35
Apparel Save                                       21
shoesforeverstoreshoesforeverstore - Ebay.com      12
DAWGS                                               8
Sears.com                                           7
Amazon.ca                                           5
autoplicity.com                                     4
UrbanInspirations                                   2
brooklynajaysbrooklynajays - Ebay.com               2
action.shackaction.shack - Ebay.com                 2
cosmic_closeouts - Ebay.com                         2
The Spot for Fits & Kicks                           2
homedepot.com                                       2
Deal Genius                 

In [340]:
df['prices.merchant'].str.contains('Walmart') == True

False

In [148]:
def return_keyword_merchant(series, keys):
    for i in range(len(keys)):
        if series.str.find(keys[i]) != -1:
            return keys[i]
    return 'other'

In [149]:
def match_merchant(column, merchant_set):
    new_merchant = []
    for i in range(len(column)):
        new_merchant.append(return_keyword(column.iloc[i], merchant_set))
    return new_merchant

In [118]:
match_category(df['categories'], category_set)

['other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'Sandals',
 'Boots',
 'other',
 'Athletic',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'Flats',
 'Flats',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boots',
 'Boo

### Colors column fixing

In [245]:
df.colors.value_counts()

Black                                                                                                                 895
Brown                                                                                                                 311
Silver                                                                                                                167
Beige                                                                                                                 159
White                                                                                                                 145
Blue                                                                                                                  139
Gray                                                                                                                  124
Green                                                                                                                 109
Black,Brown             

In [246]:
df.colors = df.colors.fillna('nan_value')

In [247]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'dateAdded',
       'dateUpdated', 'descriptions', 'dimension', 'ean', 'features',
       'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'merchants',
       'name', 'prices.amountMin', 'prices.amountMax', 'prices.availability',
       'prices.color', 'prices.condition', 'prices.currency',
       'prices.dateAdded', 'prices.dateSeen', 'prices.isSale',
       'prices.merchant', 'prices.offer', 'prices.returnPolicy',
       'prices.shipping', 'prices.size', 'prices.sourceURLs', 'quantities',
       'reviews', 'sizes', 'skus', 'sourceURLs', 'upc', 'weight'],
      dtype='object')

In [248]:
def column_split_function(df, column):
    df[column] = df[column].str.split(",\\s*").tolist()
    merge_df = df[column].apply(pd.Series)\
        .merge(df, left_index = True, right_index = True)\
        .drop([column], axis = 1)
    other_columns_set = ['id', 'asins', 'brand', 'categories', 'dateAdded',
       'dateUpdated', 'descriptions', 'dimension', 'ean', 'features',
       'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'merchants',
       'name', 'prices.amountMin', 'prices.amountMax', 'prices.availability',
       'prices.color', 'prices.condition', 'prices.currency',
       'prices.dateAdded', 'prices.dateSeen', 'prices.isSale',
       'prices.merchant', 'prices.offer', 'prices.returnPolicy',
       'prices.shipping', 'prices.size', 'prices.sourceURLs', 'quantities',
       'reviews', 'sizes', 'skus', 'sourceURLs', 'upc', 'weight']
    melt_df = merge_df.melt(id_vars = other_columns_set, value_name = column)\
        .drop("variable", axis = 1)\
        .dropna(subset = [column])\
        .reset_index()
    return melt_df

In [249]:
A = column_split_function(df, 'colors')

In [272]:
A['colors'].value_counts()[:15]

Black        2539
Brown         878
White         474
nan_value     462
Tan           425
Blue          389
Beige         388
Red           348
Grey          329
Green         307
Silver        257
Pink          210
Taupe         186
Gray          179
Gold          161
Name: colors, dtype: int64

In [260]:
df.shape

(6701, 39)

In [266]:
A.shape

(13560, 40)

In [268]:
A.drop(axis = 1, columns = ['index'], inplace = True)

In [270]:
df_after_color = A

#### We will also keep some colors in the above column, and set others into group 'other'

In [271]:
df_after_color.isnull().sum().sort_values(ascending = False)

prices.availability    13016
weight                 13007
dimension              12197
asins                  12193
prices.returnPolicy    11617
prices.shipping        10774
prices.offer            9827
reviews                 9275
manufacturer            7639
quantities              6856
prices.condition        5838
prices.size             5826
prices.color            5507
sizes                   5305
ean                     3967
skus                    3688
upc                     3391
descriptions            1666
features                1431
prices.merchant          835
merchants                535
manufacturerNumber       224
imageURLs                 16
brand                     13
categories                 0
dateAdded                  0
dateUpdated                0
colors                     0
keys                       0
name                       0
prices.amountMin           0
prices.amountMax           0
prices.currency            0
prices.dateAdded           0
prices.dateSee

### Date convert

### This method can be used to find keywords

In [273]:
def contains_date(column):
    regex_string = (r'^/(\d{4}-[01]\d-[0-3]\dT[0-2]\d:[0-5]\d:[0-5]\d\.\d+([+-][0-2]\d:[0-5]\d|Z))|(\d{4}-[01]\d-[0-3]\dT[0-2]\d:[0-5]\d:[0-5]\d([+-][0-2]\d:[0-5]\d|Z))|(\d{4}-[01]\d-[0-3]\dT[0-2]\d:[0-5]\d([+-][0-2]\d:[0-5]\d|Z))/')
    return column.str.contains(regex_string).any()

In [274]:
def find_date_features(df):
    series_contains_date = df.apply(contains_date)
    return series_contains_date.index[series_contains_date.values]

In [281]:
df = df_after_color

In [282]:
date_features = find_date_features(df)

C:\Users\51759\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [283]:
date_features

Index(['dateAdded', 'dateUpdated', 'descriptions', 'merchants',
       'prices.dateAdded', 'prices.dateSeen', 'reviews'],
      dtype='object')

In [284]:
df[date_features][:1].T

,0
dateAdded,2016-11-04T03:11:11Z
dateUpdated,2016-11-04T03:11:11Z
descriptions,"[{""dateSeen"":[""2016-11-04T03:11:11Z""],""sourceU..."
merchants,"[{""dateSeen"":[""2016-11-04T03:11:11Z""],""name"":""..."
prices.dateAdded,2016-11-04T03:11:11Z
prices.dateSeen,2016-11-03T00:00:00Z
reviews,NaN


for 'dateAdded', 'dateUpdated', 'descriptions', 'merchants','prices.dateAdded', 'prices.dateSeen', 'reviews' these 7 columns, I want to coerce and convert 'dateAdded', 'dateUpdated', 'prices.dateAdded', 'prices.dateSeen' these 4 now. Cause the 'descriptions', 'merchants', 'reviews' are not string, they are dictionary.

In [311]:
def get_first_10_chars(column_name):
    slicing_date = [w[:10] for w in df[column_name]]
    return pd.DataFrame(slicing_date).astype('datetime64[ns]')

In [313]:
get_first_10_chars('dateAdded')

(13560, 1)

In [314]:
def get_first_10_chars_for_columnSet(columnSet):
    for i in range(len(columnSet)):
        column_name = columnSet[i]
        df[column_name] = get_first_10_chars(column_name)

In [315]:
columnSet = ['dateAdded', 'dateUpdated', 'prices.dateAdded', 'prices.dateSeen']

In [316]:
get_first_10_chars_for_columnSet(columnSet)

In [319]:
df[columnSet][:2]

,dateAdded,dateUpdated,prices.dateAdded,prices.dateSeen
0,2016-11-04,2016-11-04,2016-11-04,2016-11-03
1,2016-11-04,2016-11-04,2016-11-04,2015-11-03


In [320]:
df.isnull().sum().sort_values(ascending= False)

prices.availability    13016
weight                 13007
dimension              12197
asins                  12193
prices.returnPolicy    11617
prices.shipping        10774
prices.offer            9827
reviews                 9275
manufacturer            7639
quantities              6856
prices.condition        5838
prices.size             5826
prices.color            5507
sizes                   5305
ean                     3967
skus                    3688
upc                     3391
descriptions            1666
features                1431
prices.merchant          835
merchants                535
manufacturerNumber       224
imageURLs                 16
brand                     13
categories                 0
dateAdded                  0
dateUpdated                0
colors                     0
keys                       0
name                       0
prices.amountMin           0
prices.amountMax           0
prices.currency            0
prices.dateAdded           0
prices.dateSee

In [353]:
df.shape

(13560, 39)

In [354]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'dateAdded', 'dateUpdated',
       'descriptions', 'dimension', 'ean', 'features', 'imageURLs', 'keys',
       'manufacturer', 'manufacturerNumber', 'merchants', 'name',
       'prices.amountMin', 'prices.amountMax', 'prices.availability',
       'prices.color', 'prices.condition', 'prices.currency',
       'prices.dateAdded', 'prices.dateSeen', 'prices.isSale',
       'prices.merchant', 'prices.offer', 'prices.returnPolicy',
       'prices.shipping', 'prices.size', 'prices.sourceURLs', 'quantities',
       'reviews', 'sizes', 'skus', 'sourceURLs', 'upc', 'weight', 'colors'],
      dtype='object')

### Deleting unmeaningful columns 
If I have time, I can come back to handle more columns to feed them into model.

In [400]:
df_new = df[['brand', 'categories', 'dateAdded', 'dateUpdated', 'manufacturer', 'manufacturerNumber',
    'prices.amountMin', 'prices.amountMax', 'prices.currency',
    'prices.dateAdded', 'prices.dateSeen', 'prices.isSale', 'prices.merchant', 'colors']]

In [401]:
df_new.isnull().sum().sort_values(ascending= False)

manufacturer          7639
prices.merchant        834
manufacturerNumber     224
brand                   13
colors                   0
prices.isSale            0
prices.dateSeen          0
prices.dateAdded         0
prices.currency          0
prices.amountMax         0
prices.amountMin         0
dateUpdated              0
dateAdded                0
categories               0
dtype: int64

In [402]:
df_new.shape

(13560, 14)

In [403]:
# How about drop the rows whose manufacturer is missing.
df_new.dropna(subset = ['manufacturer']).isnull().sum().sort_values(ascending= False)

prices.merchant       298
manufacturerNumber     54
brand                  13
colors                  0
prices.isSale           0
prices.dateSeen         0
prices.dateAdded        0
prices.currency         0
prices.amountMax        0
prices.amountMin        0
manufacturer            0
dateUpdated             0
dateAdded               0
categories              0
dtype: int64

In [405]:
# after drop the manufacturer missing values, there are only dozens of missing values in other columns, so I decide to drop all
# the missing values.
df_new.dropna(inplace = True)

C:\Users\51759\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [409]:
df_new.shape

(5560, 14)

In [408]:
df_new['prices.currency'].value_counts()

USD    5505
CAD      54
AUD       1
Name: prices.currency, dtype: int64

In [410]:
df_new['prices.currency'].unique()

array(['USD', 'CAD', 'AUD'], dtype=object)

### remove relative columns
I would like to use a price as my model target, and there are two price columns, including 'prices.amountMin', 'prices.amountMax'. In general, I choose the 'prices.amountMax' as target, and delete the min value, cause they are highly relative.

,brand,categories,dateAdded,dateUpdated,manufacturer,manufacturerNumber,prices.amountMax,prices.currency,prices.dateAdded,prices.dateSeen,prices.isSale,prices.merchant,colors
2,Marc Fisher,other,2016-11-11,2016-11-11,Marc Fisher,FAWNA-BLKMUL,89.00,USD,2016-11-11,2016-10-27,False,Walmart,nan_value
3,Caparros,other,2016-11-04,2016-11-04,Caparros,H1705-BLK/SLV,79.00,USD,2016-11-04,2016-10-05,False,Walmart,Silver
4,Caparros,other,2016-11-04,2016-11-04,Caparros,H1705-BLK/SLV,31.99,USD,2016-11-04,2016-11-02,True,Walmart,Silver
5,INC International Concepts,other,2016-10-27,2016-10-27,INC International Concepts,BRITANII-OLV,90.00,USD,2016-10-27,2016-10-25,False,Walmart,nan_value
6,INC International Concepts,other,2016-10-27,2016-10-27,INC International Concepts,BRITANII-OLV,35.99,USD,2016-10-27,2016-10-04,True,Walmart,nan_value
7,Vince Camuto,other,2017-01-27,2017-01-27,Vince Camuto,PARKA-MDNTTAU,159.00,USD,2017-01-27,2017-01-25,False,Walmart,Brown
8,Vince Camuto,other,2017-01-27,2017-01-27,Vince Camuto,PARKA-MDNTTAU,58.99,USD,2017-01-27,2017-01-25,True,Walmart,Brown
25,JBU by Jambu,other,2017-01-12,2017-01-12,JBU by Jambu,JB15COR34,69.00,USD,2017-01-12,2017-01-11,False,Walmart,Brown
26,JBU by Jambu,other,2017-01-12,2017-01-12,JBU by Jambu,JB15COR34,15.99,USD,2017-01-12,2017-01-11,True,Walmart,Brown
27,Style & Co.,other,2017-01-27,2017-01-27,Style & Co,VERN-KHK/PAT,19.99,USD,2017-01-27,2017-01-24,False,Walmart,Beige


In [419]:
df_new = df_new[['brand', 'manufacturer', 'manufacturerNumber', 'prices.merchant', 'categories', 'colors',
                 'dateAdded', 'dateUpdated', 'prices.dateAdded', 'prices.dateSeen',
                 'prices.currency', 'prices.isSale', 'prices.amountMax']]

In [421]:
df_new[:5]

,brand,manufacturer,manufacturerNumber,prices.merchant,categories,colors,dateAdded,dateUpdated,prices.dateAdded,prices.dateSeen,prices.currency,prices.isSale,prices.amountMax
2,Marc Fisher,Marc Fisher,FAWNA-BLKMUL,Walmart,other,nan_value,2016-11-11,2016-11-11,2016-11-11,2016-10-27,USD,False,89.00
3,Caparros,Caparros,H1705-BLK/SLV,Walmart,other,Silver,2016-11-04,2016-11-04,2016-11-04,2016-10-05,USD,False,79.00
4,Caparros,Caparros,H1705-BLK/SLV,Walmart,other,Silver,2016-11-04,2016-11-04,2016-11-04,2016-11-02,USD,True,31.99
5,INC International Concepts,INC International Concepts,BRITANII-OLV,Walmart,other,nan_value,2016-10-27,2016-10-27,2016-10-27,2016-10-25,USD,False,90.00
6,INC International Concepts,INC International Concepts,BRITANII-OLV,Walmart,other,nan_value,2016-10-27,2016-10-27,2016-10-27,2016-10-04,USD,True,35.99


### categorical columns cleaning

In [431]:
df_new.columns

Index(['brand', 'manufacturer', 'manufacturerNumber', 'prices.merchant',
       'categories', 'colors', 'dateAdded', 'dateUpdated', 'prices.dateAdded',
       'prices.dateSeen', 'prices.currency', 'prices.isSale',
       'prices.amountMax'],
      dtype='object')

In [430]:
def selected_cat_values(column, threshold = .02):
    values_counted = column.value_counts(normalize=True)
    return values_counted[values_counted > threshold]

In [437]:
selected = selected_cat_values(df_new.brand, .01)
selected.sum()
# for brand, the sum of counted values is 0.49, which is close to 0.5, it's good.

0.49082733812949647

In [438]:
selected

Nomad                       0.053777
Nike                        0.050180
Dansko                      0.035252
Nine West                   0.032734
TOMS                        0.030935
New Balance                 0.028957
Easy Spirit                 0.024281
Daniel Green                0.021043
Telic                       0.020144
Propet                      0.019424
Ugg                         0.019424
Bearpaw                     0.017986
Crocs                       0.016367
Asics                       0.013849
Charles by Charles David    0.013669
Converse                    0.013129
MICHAEL Michael Kors        0.013129
Brinley Co.                 0.012410
Alfani                      0.012410
Bloch                       0.010612
VANS                        0.010432
Nina                        0.010432
Under Armour                0.010252
Name: brand, dtype: float64

In [439]:
def reduce_cat_values(column, threshold = .02):
    column = column.copy()
    selected_values = selected_cat_values(column, threshold).index
    column[~column.isin(selected_values)] = 'other'
    column.astype('category')
    return column

In [446]:
reduce_cat_values(df_new['brand'], .01).value_counts(normalize = True)

other                       0.509173
Nomad                       0.053777
Nike                        0.050180
Dansko                      0.035252
Nine West                   0.032734
TOMS                        0.030935
New Balance                 0.028957
Easy Spirit                 0.024281
Daniel Green                0.021043
Telic                       0.020144
Propet                      0.019424
Ugg                         0.019424
Bearpaw                     0.017986
Crocs                       0.016367
Asics                       0.013849
Charles by Charles David    0.013669
Converse                    0.013129
MICHAEL Michael Kors        0.013129
Brinley Co.                 0.012410
Alfani                      0.012410
Bloch                       0.010612
VANS                        0.010432
Nina                        0.010432
Under Armour                0.010252
Name: brand, dtype: float64

In [462]:
df_new['brand'] = reduce_cat_values(df_new['brand'], .01)

In [443]:
reduce_cat_values(df_new['manufacturer'], .01).value_counts(normalize = True)

other                       0.495504
Nomad                       0.052158
Nike                        0.050180
Nine West                   0.032914
New Balance                 0.028957
Toms                        0.028417
Easy Spirit                 0.025180
Daniel Green                0.021043
DANSKO                      0.020144
Novascarpa Group LLC        0.020144
UGG Australia               0.019964
Propet                      0.019424
VANS                        0.017266
Crocs                       0.016367
Bearpaw                     0.016007
Dansko                      0.015108
asics                       0.013849
Charles by Charles David    0.013669
Converse                    0.013129
MICHAEL Michael Kors        0.013129
Alfani                      0.012410
Brinley Co                  0.012230
Pleaser Shoes               0.011511
Bloch                       0.010612
Nina                        0.010432
Under Armour                0.010252
Name: manufacturer, dtype: float64

In [463]:
df_new['manufacturer'] = reduce_cat_values(df_new['manufacturer'], .01)

* Have to say, brand and manufacturer have too many similarities, correlatonship. We will verify later.

In [445]:
df[['brand', 'manufacturer']]

,brand,manufacturer
0,BEAUTIFEET,NaN
1,BEAUTIFEET,NaN
2,Marc Fisher,Marc Fisher
3,Caparros,Caparros
4,Caparros,Caparros
5,INC International Concepts,INC International Concepts
6,INC International Concepts,INC International Concepts
7,Vince Camuto,Vince Camuto
8,Vince Camuto,Vince Camuto
9,Faded Glory,NaN


In [453]:
reduce_cat_values(df_new['manufacturerNumber'], .01).value_counts(normalize = True)

other        0.831115
W5668        0.051439
806020202    0.020144
Flip         0.020144
5825         0.019424
10007-001    0.017086
6020202      0.015108
Anna         0.013669
10004902     0.011871
Name: manufacturerNumber, dtype: float64

In [464]:
df_new['manufacturerNumber'] = reduce_cat_values(df_new['manufacturerNumber'], .01)

In [452]:
reduce_cat_values(df_new['prices.merchant'], .01).value_counts(normalize = True)

Walmart          0.753777
Overstock.com    0.192806
Shoes.com        0.027698
other            0.025719
Name: prices.merchant, dtype: float64

In [465]:
df_new['prices.merchant'] = reduce_cat_values(df_new['prices.merchant'], .01)

In [459]:
reduce_cat_values(df_new['categories'], .005).value_counts(normalize = True)

other       0.604496
Boots       0.105396
Athletic    0.092806
Sandals     0.081295
Slip-ons    0.044245
Slippers    0.039928
Sneakers    0.018345
Heels       0.007914
Flats       0.005576
Name: categories, dtype: float64

In [466]:
df_new['categories'] = reduce_cat_values(df_new['categories'], .005)

In [461]:
reduce_cat_values(df_new['colors'], .02).value_counts(normalize = True)

other         0.387590
Black         0.165108
Brown         0.075360
nan_value     0.072482
White         0.056295
Blue          0.045863
Silver        0.037410
Beige         0.034532
Gray          0.029496
Red           0.025180
Pink          0.025180
Green         0.023921
Multicolor    0.021583
Name: colors, dtype: float64

In [467]:
df_new['colors'] = reduce_cat_values(df_new['colors'], .02)

In [468]:
df_new

,brand,manufacturer,manufacturerNumber,prices.merchant,categories,colors,dateAdded,dateUpdated,prices.dateAdded,prices.dateSeen,prices.currency,prices.isSale,prices.amountMax
2,other,other,other,Walmart,other,nan_value,2016-11-11,2016-11-11,2016-11-11,2016-10-27,USD,False,89.00
3,other,other,other,Walmart,other,Silver,2016-11-04,2016-11-04,2016-11-04,2016-10-05,USD,False,79.00
4,other,other,other,Walmart,other,Silver,2016-11-04,2016-11-04,2016-11-04,2016-11-02,USD,True,31.99
5,other,other,other,Walmart,other,nan_value,2016-10-27,2016-10-27,2016-10-27,2016-10-25,USD,False,90.00
6,other,other,other,Walmart,other,nan_value,2016-10-27,2016-10-27,2016-10-27,2016-10-04,USD,True,35.99
7,other,other,other,Walmart,other,Brown,2017-01-27,2017-01-27,2017-01-27,2017-01-25,USD,False,159.00
8,other,other,other,Walmart,other,Brown,2017-01-27,2017-01-27,2017-01-27,2017-01-25,USD,True,58.99
25,other,other,other,Walmart,other,Brown,2017-01-12,2017-01-12,2017-01-12,2017-01-11,USD,False,69.00
26,other,other,other,Walmart,other,Brown,2017-01-12,2017-01-12,2017-01-12,2017-01-11,USD,True,15.99
27,other,other,other,Walmart,other,Beige,2017-01-27,2017-01-27,2017-01-27,2017-01-24,USD,False,19.99


In [479]:
# I think these 4 date type columns are highly relative to each other, I can use one of them. I choose dateAdded.
df_new = df_new.drop(columns = ['dateAdded', 'dateUpdated', 'prices.dateAdded'])

In [483]:
from date_lib import add_datepart

In [485]:
add_datepart(df_new, 'prices.dateSeen')

In [488]:
df_new[:1].T

,2
brand,other
manufacturer,other
manufacturerNumber,other
prices.merchant,Walmart
categories,other
colors,nan_value
prices.currency,USD
prices.isSale,False
prices.amountMax,89
prices.dateSeenYear,2016


In [490]:
df_new = pd.get_dummies(df_new)

In [493]:
df_new.columns.tolist()

['prices.isSale',
 'prices.amountMax',
 'prices.dateSeenYear',
 'prices.dateSeenMonth',
 'prices.dateSeenWeek',
 'prices.dateSeenDay',
 'prices.dateSeenDayofweek',
 'prices.dateSeenDayofyear',
 'prices.dateSeenIs_month_end',
 'prices.dateSeenIs_month_start',
 'prices.dateSeenIs_quarter_end',
 'prices.dateSeenIs_quarter_start',
 'prices.dateSeenIs_year_end',
 'prices.dateSeenIs_year_start',
 'prices.dateSeenElapsed',
 'brand_Alfani',
 'brand_Asics',
 'brand_Bearpaw',
 'brand_Bloch',
 'brand_Brinley Co.',
 'brand_Charles by Charles David',
 'brand_Converse',
 'brand_Crocs',
 'brand_Daniel Green',
 'brand_Dansko',
 'brand_Easy Spirit',
 'brand_MICHAEL Michael Kors',
 'brand_New Balance',
 'brand_Nike',
 'brand_Nina',
 'brand_Nine West',
 'brand_Nomad',
 'brand_Propet',
 'brand_TOMS',
 'brand_Telic',
 'brand_Ugg',
 'brand_Under Armour',
 'brand_VANS',
 'brand_other',
 'manufacturer_Alfani',
 'manufacturer_Bearpaw',
 'manufacturer_Bloch',
 'manufacturer_Brinley Co',
 'manufacturer_Charles b

In [494]:
df_new = df_new[['prices.amountMax', 'prices.isSale', 'prices.dateSeenYear', 'prices.dateSeenMonth', 'prices.dateSeenWeek',
        'prices.dateSeenDay', 'prices.dateSeenDayofweek', 'prices.dateSeenDayofyear', 'prices.dateSeenIs_month_end',
 'prices.dateSeenIs_month_start', 'prices.dateSeenIs_quarter_end', 'prices.dateSeenIs_quarter_start',
 'prices.dateSeenIs_year_end', 'prices.dateSeenIs_year_start', 'prices.dateSeenElapsed', 'brand_Alfani',
 'brand_Asics', 'brand_Bearpaw', 'brand_Bloch', 'brand_Brinley Co.', 'brand_Charles by Charles David', 'brand_Converse',
 'brand_Crocs', 'brand_Daniel Green', 'brand_Dansko', 'brand_Easy Spirit', 'brand_MICHAEL Michael Kors',
 'brand_New Balance', 'brand_Nike', 'brand_Nina', 'brand_Nine West', 'brand_Nomad', 'brand_Propet', 'brand_TOMS', 'brand_Telic',
 'brand_Ugg', 'brand_Under Armour', 'brand_VANS', 'brand_other', 'manufacturer_Alfani', 'manufacturer_Bearpaw',
 'manufacturer_Bloch', 'manufacturer_Brinley Co', 'manufacturer_Charles by Charles David', 'manufacturer_Converse',
 'manufacturer_Crocs', 'manufacturer_DANSKO', 'manufacturer_Daniel Green', 'manufacturer_Dansko', 'manufacturer_Easy Spirit',
 'manufacturer_MICHAEL Michael Kors', 'manufacturer_New Balance', 'manufacturer_Nike', 'manufacturer_Nina', 'manufacturer_Nine West',
 'manufacturer_Nomad', 'manufacturer_Novascarpa Group LLC', 'manufacturer_Pleaser Shoes', 'manufacturer_Propet',
 'manufacturer_Toms', 'manufacturer_UGG Australia', 'manufacturer_Under Armour', 'manufacturer_VANS', 'manufacturer_asics',
 'manufacturer_other', 'manufacturerNumber_10004902', 'manufacturerNumber_10007-001', 'manufacturerNumber_5825',
 'manufacturerNumber_6020202', 'manufacturerNumber_806020202', 'manufacturerNumber_Anna', 'manufacturerNumber_Flip',
 'manufacturerNumber_W5668', 'manufacturerNumber_other', 'prices.merchant_Overstock.com', 'prices.merchant_Shoes.com',
 'prices.merchant_Walmart', 'prices.merchant_other', 'categories_Athletic', 'categories_Boots', 'categories_Flats',
 'categories_Heels', 'categories_Sandals', 'categories_Slip-ons', 'categories_Slippers', 'categories_Sneakers',
 'categories_other', 'colors_Beige', 'colors_Black', 'colors_Blue', 'colors_Brown', 'colors_Gray', 'colors_Green',
 'colors_Multicolor', 'colors_Pink', 'colors_Red', 'colors_Silver', 'colors_White', 'colors_nan_value', 'colors_other',
 'prices.currency_AUD', 'prices.currency_CAD', 'prices.currency_USD']]

In [496]:
df_new.to_csv('df.csv')

In [503]:
percentiles(df_new['prices.amountMax'])

array([[ 0.        ,  0.06201976,  0.55909257,  0.26930113,  0.08964508,
         0.01994146],
       [-2.        , -1.        ,  0.        ,  1.        ,  2.        ,
         3.        ]])

In [505]:
# remove the outliers after z_score 2-3.
select_outliers(df_new['prices.amountMax']).index

Int64Index([  131,   162,   175,   188,   190,   228,   281,   315,   316,
              350,
            ...
            12936, 12970, 13109, 13110, 13119, 13213, 13214, 13223, 13323,
            13409],
           dtype='int64', length=203)

In [509]:
df_new_1 = df_new.drop(index = select_outliers(df_new['prices.amountMax']).index)

In [510]:
df_new_1.shape

(5357, 103)

In [512]:
df_new_1.to_csv('df1.csv')

In [ ]:
df_new_1